In [8]:
import torch
print(torch.cuda.is_available())  # Check if GPU is available

False


In [14]:
from huggingface_hub import hf_hub_download

print("Downloading model")
model_path = hf_hub_download(
    repo_id="TheBloke/deepseek-r1-GGUF",
    filename="deepseek-r1.Q4_K_M.gguf",
    local_dir=".",  # Saves to current directory
)
print("Downloaded")

RepositoryNotFoundError: 401 Client Error. (Request ID: Root=1-67ee6ba8-6582bb7d744c8ccd4d858a52;e3120dd0-62b8-44ff-a5c8-e817bbc4616d)

Repository Not Found for url: https://huggingface.co/TheBloke/deepseek-r1-GGUF/resolve/main/deepseek-r1.Q4_K_M.gguf.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.
Invalid username or password.

In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from datasets import load_dataset
import torch

torch.set_num_threads(8)

# Load DeepSeek-R1
model = AutoModelForCausalLM.from_pretrained(
    "deepseek-ai/deepseek-r1",       # Force CPU usage
    load_in_8bit=True,          # 8-bit quantization (lighter)
    device_map="cpu"   
)
print("Model initialised")

The repository for deepseek-ai/deepseek-r1 contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/deepseek-ai/deepseek-r1.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


RuntimeError: No GPU found. A GPU is needed for FP8 quantization.

In [ ]:
print("Start")
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-r1")

print("Loading medical dataset")
# Load medical dataset (e.g., PubMedQA)
dataset = load_dataset("pubmed_qa", "pqa_labeled")

print("Tokenizing")
# Tokenize and train
def tokenize(x):
    return tokenizer(x["question"], x["long_answer"], truncation=True)

dataset = dataset.map(tokenize, batched=True)

In [ ]:

print("Training")

# Fine-tune
training_args = TrainingArguments(
    output_dir="medical-deepseek",
    per_device_train_batch_size=4,
    num_train_epochs=3,
)

model.train()  # Add LoRA/QLoRA for efficiency
model.save_pretrained("medical-deepseek-r1")
print("Saved fined tuned medical-deepseek-r1 model")